In [6]:
# func to exec basic select
from psycopg2.extras import RealDictCursor
import psycopg2

""" Connect to the PostgreSQL database server """
conn = None

def exec_sql(sql: str):
    try:
        conn = psycopg2.connect("connect_timeout=3 dbname=tweets_db user=postgres password=root host=172.19.0.2")
        cur = conn.cursor(cursor_factory=RealDictCursor)

        cur.execute(sql)
        result = cur.fetchall()
        cur.close()
        return result
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

## uloha 1


In [7]:
sql = '''EXPLAIN SELECT * from accounts where screen_name = 'realDonaldTrump' '''
print(exec_sql(sql))

[RealDictRow([('QUERY PLAN', 'Gather  (cost=1000.00..197354.20 rows=1 width=118)')]), RealDictRow([('QUERY PLAN', '  Workers Planned: 5')]), RealDictRow([('QUERY PLAN', '  ->  Parallel Seq Scan on accounts  (cost=0.00..196354.10 rows=1 width=118)')]), RealDictRow([('QUERY PLAN', "        Filter: ((screen_name)::text = 'realDonaldTrump'::text)")]), RealDictRow([('QUERY PLAN', 'JIT:')]), RealDictRow([('QUERY PLAN', '  Functions: 2')]), RealDictRow([('QUERY PLAN', '  Options: Inlining false, Optimization false, Expressions true, Deforming true')])]


kedze nemame index nad screen_name tak pouzuje sa seq scan
planovac pouzil Parallel Seq Scan pretoze moze nad kazdym riadkom paralelne urcit ci podmienka je validna a teda ci riadok sa ma dostat do vysledku
nasledne len zozbiera vhodne riadky


| QUERY PLAN |
| :--- |
| Gather  \(cost=1000.00..223590.46 rows=1 width=119\) \(actual time=1.690..274.040 rows=1 loops=1\) |
|   Workers Planned: 2 |
|   Workers Launched: 2 |
|   -&gt;  Parallel Seq Scan on accounts  \(cost=0.00..222590.36 rows=1 width=119\) \(actual time=161.895..251.866 rows=0 loops=3\) |
|         Filter: \(\(screen\_name\)::text = 'realDonaldTrump'::text\) |
|         Rows Removed by Filter: 3229079 |
| Planning Time: 0.036 ms |
| JIT: |
|   Functions: 6 |
|   Options: Inlining false, Optimization false, Expressions true, Deforming true |


## uloha 2
na ulohe 1. pracovali 2 z 2 alokovanych workerov

nasledne budeme zvysovat `max_parallel_workers_per_gather` po 2 (default 2)

---
4/4 workers

| QUERY PLAN |
| :--- |
| Gather  \(cost=1000.00..203408.72 rows=1 width=119\) \(actual time=2.868..246.007 rows=1 loops=1\) |
|   Workers Planned: 4 |
|   Workers Launched: 4 |
|   -&gt;  Parallel Seq Scan on accounts  \(cost=0.00..202408.62 rows=1 width=119\) \(actual time=156.937..205.075 rows=0 loops=5\) |
|         Filter: \(\(screen\_name\)::text = 'realDonaldTrump'::text\) |
|         Rows Removed by Filter: 1937447 |
| Planning Time: 0.390 ms |
| JIT: |
|   Functions: 10 |
|   Options: Inlining false, Optimization false, Expressions true, Deforming true |

---
5/6 workers

| QUERY PLAN |
| :--- |
| Gather  \(cost=1000.00..197354.20 rows=1 width=119\) \(actual time=7.224..269.136 rows=1 loops=1\) |
|   Workers Planned: 5 |
|   Workers Launched: 5 |
|   -&gt;  Parallel Seq Scan on accounts  \(cost=0.00..196354.10 rows=1 width=119\) \(actual time=169.104..211.635 rows=0 loops=6\) |
|         Filter: \(\(screen\_name\)::text = 'realDonaldTrump'::text\) |
|         Rows Removed by Filter: 1614540 |
| Planning Time: 0.044 ms |
| JIT: |
|   Functions: 12 |
|   Options: Inlining false, Optimization false, Expressions true, Deforming true |

---

limit je 5 workerov pri zmene len `max_parallel_workers_per_gather` configu

nasledne som zmenil parallel_workers setting tabulky accounts na 6

| QUERY PLAN |
| :--- |
| Gather  \(cost=1000.00..193317.85 rows=1 width=119\) \(actual time=6.627..267.930 rows=1 loops=1\) |
|   Workers Planned: 6 |
|   Workers Launched: 6 |
|   -&gt;  Parallel Seq Scan on accounts  \(cost=0.00..192317.75 rows=1 width=119\) \(actual time=166.741..203.115 rows=0 loops=7\) |
|         Filter: \(\(screen\_name\)::text = 'realDonaldTrump'::text\) |
|         Rows Removed by Filter: 1383891 |
| Planning Time: 0.041 ms |
| JIT: |
|   Functions: 14 |
|   Options: Inlining false, Optimization false, Expressions true, Deforming true |

dalsie limity:

* `max_worker_processes` ktore urcuju pocet procesov pre workerow globalne
* `max_parallel_workers` limit workerov na 1 paralelnu operaciu
---
8/8 workers

tu uz je mozne vidiet zlom na mojom CPU (4c/4t) kde pocet procesov a ich sprava znizuje rychlost query

| QUERY PLAN |
| :--- |
| Gather  \(cost=1000.00..188272.41 rows=1 width=119\) \(actual time=1.869..332.442 rows=1 loops=1\) |
|   Workers Planned: 8 |
|   Workers Launched: 8 |
|   -&gt;  Parallel Seq Scan on accounts  \(cost=0.00..187272.31 rows=1 width=119\) \(actual time=208.851..244.730 rows=0 loops=9\) |
|         Filter: \(\(screen\_name\)::text = 'realDonaldTrump'::text\) |
|         Rows Removed by Filter: 1076360 |
| Planning Time: 0.066 ms |
| JIT: |
|   Functions: 18 |
|   Options: Inlining false, Optimization false, Expressions true, Deforming true |


## uloha 3

```create index accounts_screen_name on accounts USING btree (screen_name);```

| QUERY PLAN |
| :--- |
| Index Scan using accounts\_screen\_name on accounts  \(cost=0.43..8.45 rows=1 width=118\) \(actual time=0.034..0.036 rows=1 loops=1\) |
|   Index Cond: \(\(screen\_name\)::text = 'realDonaldTrump'::text\) |
| Planning Time: 0.088 ms |
| Execution Time: 0.057 ms |

cas klesol z 250/300 na 50/60 ms
kedze mame index presnych hodnot tak v nom vieme velmi rychlo zistit ktore riadky maju pozadovanu hodnotu.

wokrer bol iba 1 teda paralel index scan pouzity nebol lebo index je maly a casovo je nevyhodne vytvorit dalsi process workera

## uloha 4

```EXPLAIN ANALYSE SELECT * from public.accounts where followers_count >= 100 and followers_count <= 200;```


kedze nemame index tak sa pouzil seq scan

| QUERY PLAN |
| :--- |
| Seq Scan on accounts  \(cost=0.00..317444.57 rows=1270008 width=118\) \(actual time=3.845..1858.003 rows=1269496 loops=1\) |
|   Filter: \(\(followers\_count &gt;= 100\) AND \(followers\_count &lt;= 200\)\) |
|   Rows Removed by Filter: 8417742 |
| Planning Time: 0.137 ms |
| JIT: |
|   Functions: 2 |
|   Options: Inlining false, Optimization false, Expressions true, Deforming true |
|   Timing: Generation 0.471 ms, Inlining 0.000 ms, Optimization 0.206 ms, Emission 3.471 ms, Total 4.147 ms |
| Execution Time: 1922.475 ms |

ked som znizil cost na tuple a paralel worker init tak sa pouzil paralel seq scan ktory zlepsil cas

| QUERY PLAN |
| :--- |
| Gather  \(cost=1000.00..202197.71 rows=1270008 width=118\) \(actual time=5.253..1098.916 rows=1269496 loops=1\) |
|   Workers Planned: 5 |
|   Workers Launched: 5 |
|   -&gt;  Parallel Seq Scan on accounts  \(cost=0.00..201197.71 rows=254002 width=118\) \(actual time=8.961..815.760 rows=211583 loops=6\) |
|         Filter: \(\(followers\_count &gt;= 100\) AND \(followers\_count &lt;= 200\)\) |
|         Rows Removed by Filter: 1402957 |
| Planning Time: 0.077 ms |
| JIT: |
|   Functions: 12 |
|   Options: Inlining false, Optimization false, Expressions true, Deforming true |

v pripade ze where con. zahrna vela vysledkov tak optimalize moze uprednostnit nie paralelny pristup z dovodu velkeho costu pri presunuti udajov medzy workermi

## uloha 5

| QUERY PLAN |
| :--- |
| Bitmap Heap Scan on accounts  \(cost=17466.02..310875.89 rows=1270008 width=118\) \(actual time=135.172..2335.904 rows=1269496 loops=1\) |
|   Recheck Cond: \(\(followers\_count &gt;= 100\) AND \(followers\_count &lt;= 200\)\) |
|   Rows Removed by Index Recheck: 6449000 |
|   Heap Blocks: exact=39770 lossy=132186 |
|   -&gt;  Bitmap Index Scan on accounts\_followers\_count\_index  \(cost=0.00..17148.52 rows=1270008 width=0\) \(actual time=118.010..118.011 rows=1269496 loops=1\) |
|         Index Cond: \(\(followers\_count &gt;= 100\) AND \(followers\_count &lt;= 200\)\) |
| Planning Time: 0.178 ms |
| JIT: |
|   Functions: 2 |
|   Options: Inlining false, Optimization false, Expressions true, Deforming true |


## uloha 6

## uloha 2

## uloha 2

## uloha 2

## uloha 2

## uloha 2

## uloha 2

## uloha 2

## uloha 2

## uloha 2

## uloha 2

## uloha 2

## uloha 2

## uloha 2
